In [1]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import pymysql
from pymysql.cursors import DictCursor


In [2]:
groq_api_key = "1"
langchain_api_key = "2"
langchain_endpoint = "3"

In [3]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=2048,
    timeout=None,
    max_retries=2,
    groq_api_key=groq_api_key
)
print("Model retrieved")

Model retrieved


In [4]:
class DisasterClassification(BaseModel):
    disaster_type: str = Field(
        description="Classify the type of disaster. Possible types include 'flood', 'earthquake', 'fire', etc. Return only the disaster type."
    )
    severity: int = Field(
        description="Classify the severity of the disaster on a scale from 0 to 3. 0 being no impact and 3 being extremely severe."
    )

# Create structured LLM grader
structured_llm_grader = llm.with_structured_output(DisasterClassification)

# Define system prompt for disaster classification
system_prompt = """You are an expert in disaster management. Given the description of a disaster, classify it into one of the following types: 'flood', 'earthquake', 'fire', etc. 
    Additionally, assess the severity of the disaster on a scale from 0 to 3, where 0 indicates no impact and 3 indicates extremely severe. 
    Provide accurate and detailed classifications based on the given description."""

# Create the prompt template
disaster_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Disaster description: {query}")
    ]
)

# Combine prompt template with structured output
disaster_classifier = disaster_prompt | structured_llm_grader

In [5]:
query = "a big earthquake in 5 km radius destroying everything in its path"
disaster_info = disaster_classifier.invoke({"query": query})

disaster_type = disaster_info.disaster_type
severity = disaster_info.severity

print(f"Disaster Type: {disaster_type}")
print(f"Severity: {severity}")

Disaster Type: earthquake
Severity: 3


In [6]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '4',
    'database': 'sihdb'  # Replace with your actual database name
}


In [7]:
last_processed_id = 3

def process_alerts():
    global last_processed_id

    # Connect to the database
    connection = pymysql.connect(**db_config)

    try:
        with connection.cursor(DictCursor) as cursor:
            # Retrieve unprocessed data from Alert table starting from the last processed ID
            sql = "SELECT * FROM Alert WHERE id > %s"
            cursor.execute(sql, (last_processed_id,))
            alerts = cursor.fetchall()

            for alert in alerts:
                description = alert['description']
                
                # Predict disaster type and urgency
                disaster_info = disaster_classifier.invoke({"query": description})
                disaster_type = disaster_info.disaster_type
                severity = disaster_info.severity
                
                # Prepare data for the new table
                ndrf_data = {
                    'alert_id': alert['id'],
                    'user_name': alert['name'],
                    'mobile_number': alert['mobile_number'],
                    'state_name': alert['state_name'],
                    'city_name': alert['city_name'],
                    'pincode': alert['pincode'],
                    'urgency': severity,
                    'disaster_type': disaster_type
                }

                # Insert data into the NDRF table
                cursor.execute("""
                    INSERT INTO NDRFTasks (alert_id, user_name, mobile_number, state_name, city_name, pincode, urgency, disaster_type)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (ndrf_data['alert_id'], ndrf_data['user_name'], ndrf_data['mobile_number'],
                      ndrf_data['state_name'], ndrf_data['city_name'], ndrf_data['pincode'],
                      ndrf_data['urgency'], ndrf_data['disaster_type']))

                # Update the last processed alert ID
                last_processed_id = alert['id']

        # Commit changes
        connection.commit()

    finally:
        connection.close()

In [8]:
if __name__ == "__main__":
    process_alerts()

In [1]:
print("ok")

ok
